# Transfer Learning con MobileNetV2
## Clasificación Binaria Seleccionable

Este cuaderno permite elegir uno de los siguientes datasets binarios integrados en `tensorflow_datasets` y entrenar un clasificador basado en MobileNetV2:

1. **horses_or_humans**
2. **cats_vs_dogs**
3. **smile** (atributo *Smiling* del dataset CelebA)
4. **covid_chestxray** (radiografías de tórax normales vs neumonía/COVID‑19)

> Selecciona el dataset en la celda siguiente y ejecuta todo el notebook para entrenar, evaluar la precisión y probar con imágenes propias.

In [ ]:
!pip install -q tensorflow tensorflow-datasets ipywidgets

In [ ]:
#@title Selección de dataset { run: 'auto' }
DATASET_NAME = 'horses_or_humans' #@param ['horses_or_humans', 'cats_vs_dogs', 'smile', 'covid_chestxray']
IMG_SIZE = 160
BATCH_SIZE = 32
EPOCHS = 5

In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt
print(f'Dataset seleccionado: {DATASET_NAME}')

In [ ]:
def load_dataset(name):
    if name == 'horses_or_humans':
        splits = ['train[:70%]', 'train[70%:85%]', 'train[85%:]']
        train_ds, val_ds, test_ds = tfds.load(
            'horses_or_humans',
            split=splits,
            as_supervised=True,
        )
    elif name == 'cats_vs_dogs':
        splits = ['train[:80%]', 'train[80%:90%]', 'train[90%:]']
        train_ds, val_ds, test_ds = tfds.load(
            'cats_vs_dogs',
            split=splits,
            as_supervised=True,
        )
    elif name == 'smile':
        # CelebA: usamos el atributo 'Smiling' como etiqueta binaria
        splits = ['train[:80%]', 'train[80%:90%]', 'train[90%:]']
        def map_smile(example):
            image = example['image']
            label = example['attributes']['Smiling']
            return image, label
        train_ds = tfds.load('celeb_a', split=splits[0]).map(map_smile)
        val_ds   = tfds.load('celeb_a', split=splits[1]).map(map_smile)
        test_ds  = tfds.load('celeb_a', split=splits[2]).map(map_smile)
    else:  # covid_chestxray
        splits = ['train', 'validation', 'test']
        train_ds, val_ds, test_ds = tfds.load(
            'chest_xray',
            split=splits,
            as_supervised=True,
        )
    return train_ds, val_ds, test_ds

In [ ]:
train_ds, val_ds, test_ds = load_dataset(DATASET_NAME)
print(train_ds, val_ds, test_ds)

In [ ]:
def preprocess(image, label):
    image = tf.image.resize(image, (IMG_SIZE, IMG_SIZE))
    image = tf.cast(image, tf.float32) / 255.0
    return image, label

train_ds = train_ds.map(preprocess).shuffle(1000).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
val_ds   = val_ds.map(preprocess).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
test_ds  = test_ds.map(preprocess).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

In [ ]:
base_model = tf.keras.applications.MobileNetV2(
    weights='imagenet', include_top=False, input_shape=(IMG_SIZE, IMG_SIZE, 3))
base_model.trainable = False

inputs = tf.keras.Input(shape=(IMG_SIZE, IMG_SIZE, 3))
x = base_model(inputs, training=False)
x = tf.keras.layers.GlobalAveragePooling2D()(x)
outputs = tf.keras.layers.Dense(1, activation='sigmoid')(x)
model = tf.keras.Model(inputs, outputs)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

In [ ]:
history = model.fit(train_ds, epochs=EPOCHS, validation_data=val_ds)

In [ ]:
test_loss, test_acc = model.evaluate(test_ds)
print(f'Precisión en datos de prueba: {test_acc:.3f}')

In [ ]:
plt.figure()
plt.plot(history.history['accuracy'], label='Entrenamiento')
plt.plot(history.history['val_accuracy'], label='Validación')
plt.legend()
plt.title('Precisión por época')
plt.show()

In [ ]:
from google.colab import files
uploaded = files.upload()

for fname in uploaded.keys():
    img = tf.keras.preprocessing.image.load_img(fname, target_size=(IMG_SIZE, IMG_SIZE))
    img_array = tf.keras.preprocessing.image.img_to_array(img)
    img_array = tf.expand_dims(img_array, 0) / 255.0
    prediction = model.predict(img_array)[0][0]
    label = 'Positivo' if prediction > 0.5 else 'Negativo'
    print(f"{fname}: {label} ({prediction:.2f})")